https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP#scrollTo=VJ76KiP_dLn-

https://colab.research.google.com/drive/1uSlWtJdZmLrI3FCNIlUHFxwAJiSu2J0-

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools
import math
from torch.jit import script, trace
from torch import optim

print(torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

1.7.1+cu110
cuda


In [2]:
import tensorflow as tf
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer # kobert의 get_tokenizer와 같이 쓰기 위함
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [3]:
corpus_name = '한국어 연속 대화 데이터셋'
corpus = os.path.join(r'C:\Users\abc\Desktop', corpus_name)

def printLines(file, n=15):
    with open(file, 'r', encoding='cp949') as datafile:
        next(datafile)
        next(datafile)
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)
        
printLines(os.path.join(corpus, '한국어_연속적_대화_데이터셋.txt'))

S	아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!	분노									

	그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.	혐오									

	손님 왔어요.	중립									

	손님? 누구?	중립									

	몰라요. 팀장님 친구래요.	중립									

	내 친구? 친구 누구?	중립									

	그걸 내가 어떻게 알아요!	분노									

S	그래서... 무슨 일 해?	중립									

	그냥 방송일 조금. 	중립									

	방송? 방송 뭐?	중립									

	케이블 같은데서 아침에 배도 타고 산도 오르고. 있어 그런 거.	중립									

	사는 덴 어디야?	중립									

	개포동.	중립									

	남편은 뭐하는데?	중립									

	동사무소에서 인구조사 나왔니? 지금.	분노									



In [4]:
def loadLines(fileName):
    change_list = []
    temp_list = []
    lines = []
    count = 0
    count_list = []
    with open(fileName, 'r', encoding='cp949') as f:
        next(f)
        next(f)
        for line in f:
            change_list = line.strip().split('\t')

            if change_list[0] == 'S':
                count_list.append(count)
                count = 0
                temp_list.append(change_list[1])
            else:
                count += 1
                temp_list.append(change_list[0])
                
        count_list.append(count) # 마지막 대화문 카운트 입력

        count = 0
        for i in count_list[1:]: # 6, 7 ,13, 11
            for j in range(count+1, count+i+1):
                lines.append(temp_list[j - 1] + '[SEP]' + temp_list[j])
            count = count + i + 1 # 한 대화가 끝날때마다 변해야할 변수가 필요하므로 다음대화 시작하기 위해 카운트 1증가하여 구분
                
    return lines

lines = loadLines(os.path.join(corpus, '한국어_연속적_대화_데이터셋.txt'))
lines[:30]

['아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데![SEP]그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.',
 '그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.[SEP]손님 왔어요.',
 '손님 왔어요.[SEP]손님? 누구?',
 '손님? 누구?[SEP]몰라요. 팀장님 친구래요.',
 '몰라요. 팀장님 친구래요.[SEP]내 친구? 친구 누구?',
 '내 친구? 친구 누구?[SEP]그걸 내가 어떻게 알아요!',
 '그래서... 무슨 일 해?[SEP]그냥 방송일 조금. ',
 '그냥 방송일 조금. [SEP]방송? 방송 뭐?',
 '방송? 방송 뭐?[SEP]케이블 같은데서 아침에 배도 타고 산도 오르고. 있어 그런 거.',
 '케이블 같은데서 아침에 배도 타고 산도 오르고. 있어 그런 거.[SEP]사는 덴 어디야?',
 '사는 덴 어디야?[SEP]개포동.',
 '개포동.[SEP]남편은 뭐하는데?',
 '남편은 뭐하는데?[SEP]동사무소에서 인구조사 나왔니? 지금.',
 '언제부터 시작할 수 있어?[SEP]나? ... 나보고 하라고?',
 '나? ... 나보고 하라고?[SEP]그럼 내가 널 왜 만나러 왔겠어?',
 '그럼 내가 널 왜 만나러 왔겠어?[SEP]근데 왜... 나한테...?',
 '근데 왜... 나한테...?[SEP]"내가 몇 명 만나봤는데, 보니까 죄다 사기꾼 같고 영 못 믿겠어. 아무래도 이런 일은 아는 사람한테 맡기는 게... 딴 사람들도 다 그러더라."',
 '"내가 몇 명 만나봤는데, 보니까 죄다 사기꾼 같고 영 못 믿겠어. 아무래도 이런 일은 아는 사람한테 맡기는 게... 딴 사람들도 다 그러더라."[SEP]에이. 내가 무슨. 나 못해.',
 '에이. 내가 무슨. 나 못해.[SEP]왜?',
 '왜?[SEP]나 이런 거 한번도 안 해봤어.',
 '나 이런 거 한번도 안 해봤어.[SEP]잘됐네. 그럼 이번에 해보면 되잖아.',
 '잘됐네. 그럼 이번에 해보면 되잖아.

In [5]:
datafile = os.path.join(corpus, '한국어_연속적_대화_데이터셋_처리후.txt')

delimiter = '\t'
delimiter = str(codecs.decode(delimiter, 'unicode_escape'))

# 띄어쓰기가 2번 입력됨
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='cp949') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n', quoting=csv.QUOTE_NONE, escapechar=' ')
    #writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in lines:
        writer.writerow([pair])

print("\nSample lines from file:")
printLines(datafile, n=3)

# 띄어쓰기 2번을 1번으로 바꿔서 다시 저장
print("\nRewriting newly formatted file...")
with open(datafile, 'w', encoding='cp949') as outputfile:
    for pair in lines:
        outputfile.write(pair.replace('  ', ' '))
        outputfile.write('\n')
        
print("\nSample lines from file:")
printLines(datafile, n=3)


Writing newly formatted file...

Sample lines from file:
손님  왔어요.[SEP]손님?  누구?

손님?  누구?[SEP]몰라요.  팀장님  친구래요.

몰라요.  팀장님  친구래요.[SEP]내  친구?  친구  누구?


Rewriting newly formatted file...

Sample lines from file:
손님 왔어요.[SEP]손님? 누구?

손님? 누구?[SEP]몰라요. 팀장님 친구래요.

몰라요. 팀장님 친구래요.[SEP]내 친구? 친구 누구?



In [6]:
model, vocab  = get_pytorch_kobert_model()

using cached model
using cached model


In [7]:
PAD_token = 0
SOS_token = 1
EOS_token = 2
CLS_token = 101
SEP_token = 102
tokenizer = get_tokenizer() 
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
transform = nlp.data.BERTSentenceTransform(tokenizer=tok, max_seq_length=64, pad=False, pair=False)

class Voc:
    def __init__(self, name):
        self.name = name
        self.tokens = []
        
    def addSentence(self, sentence): # sentence : 잘 마실게
        tokens = tok(sentence)
        self.tokens = tokens

        return tokens            

using cached model


In [8]:
MAX_LENGTH = 15 

def readVocs(datafile, corpus_name): # corpus_name : chatData / datafile : formatted_ko_conversations.txt
    print('Reading lines...')
    lines = open(datafile, encoding='cp949').read().strip().split('\n')
    pairs = [l.strip().split('[SEP]') for l in lines]
    #print(pairs[0])
    voc = Voc(corpus_name)
    return voc, pairs # voc : 문서 단어집합 / pairs : 문장 쌍 집합

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name) # voc : 단어집합, pairs : 질문 쌍
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    for pair in pairs:
        sentence1 = voc.addSentence(pair[0].strip())
        #print(sentence1)
        sentence2 = voc.addSentence(pair[1].strip())
    return voc, pairs

save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)

for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 45687 sentence pairs
Trimmed to 38509 sentence pairs
['아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!', '그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.']
['그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.', '손님 왔어요.']
['손님 왔어요.', '손님? 누구?']
['손님? 누구?', '몰라요. 팀장님 친구래요.']
['몰라요. 팀장님 친구래요.', '내 친구? 친구 누구?']
['내 친구? 친구 누구?', '그걸 내가 어떻게 알아요!']
['그래서... 무슨 일 해?', '그냥 방송일 조금.']
['그냥 방송일 조금. ', '방송? 방송 뭐?']
['방송? 방송 뭐?', '케이블 같은데서 아침에 배도 타고 산도 오르고. 있어 그런 거.']
['케이블 같은데서 아침에 배도 타고 산도 오르고. 있어 그런 거.', '사는 덴 어디야?']


In [9]:
def indexesFromSentence(voc, sentence):
    token_to_encode = transform([sentence]) # token_to_encode 튜플형식으로 반환
    return token_to_encode[0].tolist() # 0번째를 추출하여 리스트형식으로 변환해서 반환

def zeroPadding(l, fillvalue=PAD_token):
    # print(list(itertools.zip_longest(*l, fillvalue=fillvalue))) : (1957, 2, 2, 2, 0)
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

def inputVar(l, voc): # l : 응. 새벽에도 일찍 나갔어. 온다간다 말도없이.
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l] 
    #print(indexes_batch) # : [[5027, 1239, 9433, 2], [5951, 4686, 1476, 2], [1116, 5309, 2], [319, 2], [186, 2]]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch]) 
    # print(lengths) : tensor([4, 4, 3, 2, 2])
    padList = zeroPadding(indexes_batch)
    # print(padList) : [[5027, 1239, 9433, 2], [5951, 4686, 1476, 2], [1116, 5309, 2, 0], [319, 2, 0, 0], [186, 2, 0, 0]]
    padVar = torch.LongTensor(padList)
    return padVar, lengths

def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch]) 
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList) 
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(transform([x[0]])[0].tolist()), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        #print(pair[0])
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[   2,    2,    2,    2,    2],
        [3093,  517, 3220, 3222, 5176],
        [ 862, 5739, 6179, 2801,  517],
        [7398,  517,  517, 7034,   55],
        [ 517,  463,    5,  633,    3],
        [  54, 1457, 4102,    3,    0],
        [2563, 3036,  633,    0,    0],
        [3219, 6733, 3245,    0,    0],
        [7848, 7828,  633,    0,    0],
        [ 517, 2627,    3,    0,    0],
        [  54, 4400,    0,    0,    0],
        [4067, 5561,    0,    0,    0],
        [7227, 4012,    0,    0,    0],
        [7120, 5591,    0,    0,    0],
        [ 517, 7344,    0,    0,    0],
        [  54, 3135,    0,    0,    0],
        [1195, 5330,    0,    0,    0],
        [3225,  517,    0,    0,    0],
        [ 517,  463,    0,    0,    0],
        [5878,    3,    0,    0,    0],
        [6855,    0,    0,    0,    0],
        [ 633,    0,    0,    0,    0],
        [   3,    0,    0,    0,    0]])
lengths: tensor([23, 20, 10,  6,  5])
target_variable: tensor([

In [10]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, 
                          dropout = (0 if n_layers == 1 else dropout), bidirectional=True)
        
    def forward(self, input_seq, input_lengths, hidden=None):
        embedded = self.embedding(input_seq) # input_seq : shape=(max_length, batch_size), input_variable
        # print(input_lengths) : =lengths
        # print(embedded.shape) : torch.Size([10, 64, 500]) [max_length, batch_size, hidden_size(은닉상태 크기)]
        
        # nn.utils.rnn.pack_padded_sequence : 패딩연산처리 쉽게하기 위해 중간에 빈공간 제거(형태 : tensor)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths.cpu()) # input_lengths : shape=(batch_size)
        #print(packed.batch_sizes)
        # print(packed.batch_sizes) : tensor([64, 64, 64, 58, 52, 45, 38, 17,  8,  2])
        
        outputs, hidden = self.gru(packed, hidden) # 입력hidden : shape=(n_layers * num_directions, batch_size, hidden_size)
        # print(outputs.batch_sizes) : tensor([64, 64, 63, 52, 47, 34, 24, 18, 12,  6])
        # print(hidden.shape) : torch.Size([4, 64, 500]) [층 * 양방향이면2 아니면1, batch_size, hidden_size]
        
        # nn.utils.rnn.pad_packed_sequence : 패딩연산이 끝난 것을 다시 원래대로 (형태 : torch)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        #print(outputs.shape)
        # print(outputs.shape)# : torch.Size([10, 64, 1000]) # [max_length, batch_size, hidden_size(양방향으로 진행했으면 *2)]
        
        # 양방향 GRU의 출력을 합산합니다
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # print(outputs.shape) : torch.Size([10, 64, 500])
        
        # hidden : GRU의 최종 은닉 상태
        return outputs, hidden

In [11]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, 'is not an appropriate attention method.')
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))
            
    # 가중치 계산을 dot-product로 계산
    def dot_score(self, hidden, encoder_output):
        # print(torch.sum(hidden * encoder_output, dim=2).shape) : torch.Size([10, 64]) 10개 생성[max_length, batch_size]
        return torch.sum(hidden * encoder_output, dim=2)
    
    # 
    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        # print(energy.shape) : torch.Size([10, 64, 500]) 10개 생성[max_length, batch_size, hidden_size]
        
        # print(torch.sum(hidden * energy, dim=2).shape) : torch.Size([10, 64]) 10개 생성[max_length, batch_size]
        return torch.sum(hidden * energy, dim=2)
    
    
    def concat_score(self, hidden, encoder_output):
        # cat : 합칠 때 차원은 2차원으로 / expand : 확장
        # Tanh 함수는 함수값을 [-1, 1]로 제한시킴
        # print((hidden.expand(encoder_output.size(0), -1, -1).shape)) : torch.Size([10, 64, 500])
        # print(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2).shape) : torch.Size([10, 64, 1000])
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        # print(energy.shape) : torch.Size([10, 64, 500]) 10개 생성[max_length, batch_size, hidden_size]
        return torch.sum(self.v * energy, dim=2)
    
    def forward(self, hidden, encoder_outputs):
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)
            
        attn_energies = attn_energies.t() # t() : 행과 열을 바꿔서 저장[1, 2, 3], [4, 5, 6] -> [1, 4, 7], [2, 5, 8]
        
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [12]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # 참조를 보존해 둡니다
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # 레이어를 정의합니다
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # 주의: 한 단위 시간에 대해 한 단계(단어)만을 수행합니다
        # 현재의 입력 단어에 대한 임베딩을 구합니다   
        #print(input_step)
        embedded = self.embedding(input_step) # input_step : 입력 시퀀스 배치에 대한 한 단위 시간(한 단어). shape=(1, batch_size)
        embedded = self.embedding_dropout(embedded)
        #print(embedded.shape)
        # print(embedded.shape) : torch.Size([1, 64, 500])
        
        # 양방향x
        # last_hidden : GRU의 마지막 은닉 레이어. shape=(n_layers * num_directions, batch_size, hidden_size)
        # print(last_hidden.shape) : torch.Size([2, 64, 500]) 
        rnn_output, hidden = self.gru(embedded, last_hidden) 
        # print(rnn_output.shape) : torch.Size([1, 64, 500])
        # print(hidden.shape) : torch.Size([2, 64, 500])

        # attention 가중치
        attn_weights = self.attn(rnn_output, encoder_outputs) # encoder_outputs : 인코더 모델 출력 shape=(max_length, batch_size, hidden_size)
        # print(attn_weights.shape) : torch.Size([64, 1, 10]) 

        # 인코더 출력에 어텐션을 곱하여 새로운 context vector생성
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # print(context.shape) : torch.Size([64, 1, 500])

        rnn_output = rnn_output.squeeze(0) # print(rnn_output.shape) : torch.Size([64, 500])
        context = context.squeeze(1) # print(context.shape) : torch.Size([64, 500])
        concat_input = torch.cat((rnn_output, context), 1) # print(concat_input.shape) : torch.Size([64, 1000])
        concat_output = torch.tanh(self.concat(concat_input))
        # print(concat_output.shape) : torch.Size([64, 500])

        # output : 각 단어가 디코딩된 시퀀스에서 다음 단어로 사용되었을 때 적합할 확률을 나타내는 정규화된 softmax 텐서. 
        # shape=(batch_size, voc.num_words)
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)

        return output, hidden

In [13]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [14]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding, encoder_optimizer, decoder_optimizer,
         batch_size, clip, max_length = MAX_LENGTH):
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    
    loss = 0
    print_losses = []
    n_totals = 0
    
    # EncoderRNN의 forward부분 실행
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
    
    # 초기 디코더 입력을 생성(각 문장을 SOS 토큰으로 시작)
    decoder_input = torch.LongTensor([[CLS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)
    
    # 디코더의 초기 은닉 상태를 인코더의 마지막 은닉 상태로
    decoder_hidden = encoder_hidden[:decoder.n_layers]
    
    # teacher_forcing : Decoder부분에서 앞 단어가 잘못 추측되었을 경우 뒤에도 달라지니 정답을 입력해 주는 것
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        for t in range(max_target_len):
            # LuongAttnDecoderRNN의 forward로 실행
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            
             # Teacher forcing 사용: 다음 입력을 현재의 목표로 둡니다
            decoder_input = target_variable[t].view(1, -1)
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            
            # Teacher forcing 미사용: 다음 입력을 디코더의 출력으로 둡니다
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
            
    loss.backward()
    
    # clip_grad_norm_: 그라디언트를 제자리에서 수정합니다
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return sum(print_losses) / n_totals

In [15]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # 각 단계에 대한 배치 설정
    # batch2TrainData : return inp, lengths, output, mask, max_target_len
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]


    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1


    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        torch.cuda.empty_cache() # GPU 캐시 데이터 삭제
        training_batch = training_batches[iteration - 1]
        
        input_variable, lengths, target_variable, mask, max_target_len = training_batch


        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        
        print_loss += loss


        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Checkpoint를 저장
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [16]:
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 32

loadFilename = None
checkpoint_iter = 20000
loadFilename = os.path.join(save_dir, model_name, corpus_name,
                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
                            '{}_checkpoint.tar'.format(checkpoint_iter))


# loadFilename이 제공되는 경우에는 모델을 불러옵니다
if loadFilename:
    # 모델을 학습할 때와 같은 기기에서 불러오는 경우
    checkpoint = torch.load(loadFilename)
    # GPU에서 학습한 모델을 CPU로 불러오는 경우
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')

embedding = nn.Embedding(len(vocab), hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, len(vocab), decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)

encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [ ]:
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 40000
print_every = 1
save_every = 10000

# Dropout 레이어를 학습 모드로 둡니다
encoder.train()
decoder.train()

# Optimizer를 초기화합니다
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)


for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# 학습 단계를 수행합니다
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...


<ipython-input-13-f319a975feb5>:4: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen/native/IndexingUtils.h:25.)
  loss = crossEntropy.masked_select(mask).mean()


Iteration: 20001; Percent complete: 50.0%; Average loss: 2.4137
Iteration: 20002; Percent complete: 50.0%; Average loss: 2.2842
Iteration: 20003; Percent complete: 50.0%; Average loss: 2.2198
Iteration: 20004; Percent complete: 50.0%; Average loss: 2.0357
Iteration: 20005; Percent complete: 50.0%; Average loss: 2.1100
Iteration: 20006; Percent complete: 50.0%; Average loss: 1.8983
Iteration: 20007; Percent complete: 50.0%; Average loss: 2.1984
Iteration: 20008; Percent complete: 50.0%; Average loss: 2.2308
Iteration: 20009; Percent complete: 50.0%; Average loss: 2.1423
Iteration: 20010; Percent complete: 50.0%; Average loss: 2.2688
Iteration: 20011; Percent complete: 50.0%; Average loss: 1.9262
Iteration: 20012; Percent complete: 50.0%; Average loss: 1.9749
Iteration: 20013; Percent complete: 50.0%; Average loss: 2.2175
Iteration: 20014; Percent complete: 50.0%; Average loss: 1.7861
Iteration: 20015; Percent complete: 50.0%; Average loss: 2.3167
Iteration: 20016; Percent complete: 50.0

Iteration: 20130; Percent complete: 50.3%; Average loss: 2.1065
Iteration: 20131; Percent complete: 50.3%; Average loss: 2.2500
Iteration: 20132; Percent complete: 50.3%; Average loss: 2.1879
Iteration: 20133; Percent complete: 50.3%; Average loss: 2.1207
Iteration: 20134; Percent complete: 50.3%; Average loss: 2.0945
Iteration: 20135; Percent complete: 50.3%; Average loss: 2.2687
Iteration: 20136; Percent complete: 50.3%; Average loss: 2.1204
Iteration: 20137; Percent complete: 50.3%; Average loss: 2.2709
Iteration: 20138; Percent complete: 50.3%; Average loss: 2.1621
Iteration: 20139; Percent complete: 50.3%; Average loss: 2.1090
Iteration: 20140; Percent complete: 50.3%; Average loss: 2.2938
Iteration: 20141; Percent complete: 50.4%; Average loss: 2.1744
Iteration: 20142; Percent complete: 50.4%; Average loss: 2.1457
Iteration: 20143; Percent complete: 50.4%; Average loss: 2.1117
Iteration: 20144; Percent complete: 50.4%; Average loss: 2.2814
Iteration: 20145; Percent complete: 50.4

Iteration: 20259; Percent complete: 50.6%; Average loss: 2.0497
Iteration: 20260; Percent complete: 50.6%; Average loss: 2.3315
Iteration: 20261; Percent complete: 50.7%; Average loss: 2.2466
Iteration: 20262; Percent complete: 50.7%; Average loss: 2.1685
Iteration: 20263; Percent complete: 50.7%; Average loss: 2.0666
Iteration: 20264; Percent complete: 50.7%; Average loss: 2.2094
Iteration: 20265; Percent complete: 50.7%; Average loss: 2.0586
Iteration: 20266; Percent complete: 50.7%; Average loss: 1.8851
Iteration: 20267; Percent complete: 50.7%; Average loss: 2.0406
Iteration: 20268; Percent complete: 50.7%; Average loss: 2.0767
Iteration: 20269; Percent complete: 50.7%; Average loss: 2.4133
Iteration: 20270; Percent complete: 50.7%; Average loss: 2.2830
Iteration: 20271; Percent complete: 50.7%; Average loss: 2.1890
Iteration: 20272; Percent complete: 50.7%; Average loss: 1.7065
Iteration: 20273; Percent complete: 50.7%; Average loss: 2.3666
Iteration: 20274; Percent complete: 50.7

...

Iteration: 29934; Percent complete: 74.8%; Average loss: 1.8838
Iteration: 29935; Percent complete: 74.8%; Average loss: 1.5931
Iteration: 29936; Percent complete: 74.8%; Average loss: 1.9749
Iteration: 29937; Percent complete: 74.8%; Average loss: 1.9196
Iteration: 29938; Percent complete: 74.8%; Average loss: 1.7215
Iteration: 29939; Percent complete: 74.8%; Average loss: 1.8373
Iteration: 29940; Percent complete: 74.9%; Average loss: 1.8037
Iteration: 29941; Percent complete: 74.9%; Average loss: 1.7841
Iteration: 29942; Percent complete: 74.9%; Average loss: 1.6421
Iteration: 29943; Percent complete: 74.9%; Average loss: 1.7564
Iteration: 29944; Percent complete: 74.9%; Average loss: 1.9423
Iteration: 29945; Percent complete: 74.9%; Average loss: 1.7451
Iteration: 29946; Percent complete: 74.9%; Average loss: 1.6631
Iteration: 29947; Percent complete: 74.9%; Average loss: 1.7710
Iteration: 29948; Percent complete: 74.9%; Average loss: 1.8752
Iteration: 29949; Percent complete: 74.9

Iteration: 30063; Percent complete: 75.2%; Average loss: 1.8204
Iteration: 30064; Percent complete: 75.2%; Average loss: 1.5893
Iteration: 30065; Percent complete: 75.2%; Average loss: 1.5258
Iteration: 30066; Percent complete: 75.2%; Average loss: 1.4719
Iteration: 30067; Percent complete: 75.2%; Average loss: 1.5789
Iteration: 30068; Percent complete: 75.2%; Average loss: 1.8451
Iteration: 30069; Percent complete: 75.2%; Average loss: 1.7322
Iteration: 30070; Percent complete: 75.2%; Average loss: 1.8284
Iteration: 30071; Percent complete: 75.2%; Average loss: 2.0065
Iteration: 30072; Percent complete: 75.2%; Average loss: 1.6307
Iteration: 30073; Percent complete: 75.2%; Average loss: 1.8006
Iteration: 30074; Percent complete: 75.2%; Average loss: 1.4995
Iteration: 30075; Percent complete: 75.2%; Average loss: 1.8627
Iteration: 30076; Percent complete: 75.2%; Average loss: 1.7342
Iteration: 30077; Percent complete: 75.2%; Average loss: 2.0803
Iteration: 30078; Percent complete: 75.2

Iteration: 30192; Percent complete: 75.5%; Average loss: 1.5626
Iteration: 30193; Percent complete: 75.5%; Average loss: 1.9265
Iteration: 30194; Percent complete: 75.5%; Average loss: 1.6230
Iteration: 30195; Percent complete: 75.5%; Average loss: 1.4987
Iteration: 30196; Percent complete: 75.5%; Average loss: 1.7951
Iteration: 30197; Percent complete: 75.5%; Average loss: 1.7165
Iteration: 30198; Percent complete: 75.5%; Average loss: 1.8221
Iteration: 30199; Percent complete: 75.5%; Average loss: nan
Iteration: 30200; Percent complete: 75.5%; Average loss: 1.6205
Iteration: 30201; Percent complete: 75.5%; Average loss: 1.7233
Iteration: 30202; Percent complete: 75.5%; Average loss: 1.6103
Iteration: 30203; Percent complete: 75.5%; Average loss: 1.9317
Iteration: 30204; Percent complete: 75.5%; Average loss: 1.7328
Iteration: 30205; Percent complete: 75.5%; Average loss: 1.7482
Iteration: 30206; Percent complete: 75.5%; Average loss: 1.8691
Iteration: 30207; Percent complete: 75.5%; 

Iteration: 30321; Percent complete: 75.8%; Average loss: 1.4797
Iteration: 30322; Percent complete: 75.8%; Average loss: 1.6157
Iteration: 30323; Percent complete: 75.8%; Average loss: 1.8094
Iteration: 30324; Percent complete: 75.8%; Average loss: 1.7011
Iteration: 30325; Percent complete: 75.8%; Average loss: 1.8616
Iteration: 30326; Percent complete: 75.8%; Average loss: 1.6964
Iteration: 30327; Percent complete: 75.8%; Average loss: 1.6127
Iteration: 30328; Percent complete: 75.8%; Average loss: 1.7470
Iteration: 30329; Percent complete: 75.8%; Average loss: 1.7834
Iteration: 30330; Percent complete: 75.8%; Average loss: 1.7188
Iteration: 30331; Percent complete: 75.8%; Average loss: 1.6756
Iteration: 30332; Percent complete: 75.8%; Average loss: 1.5832
Iteration: 30333; Percent complete: 75.8%; Average loss: 1.7037
Iteration: 30334; Percent complete: 75.8%; Average loss: 1.6498
Iteration: 30335; Percent complete: 75.8%; Average loss: 1.8025
Iteration: 30336; Percent complete: 75.8

Iteration: 30450; Percent complete: 76.1%; Average loss: 1.8110
Iteration: 30451; Percent complete: 76.1%; Average loss: 1.9654
Iteration: 30452; Percent complete: 76.1%; Average loss: 1.6337
Iteration: 30453; Percent complete: 76.1%; Average loss: 1.6654
Iteration: 30454; Percent complete: 76.1%; Average loss: 1.9878
Iteration: 30455; Percent complete: 76.1%; Average loss: 1.8290


In [ ]:
# 탐욕적 디코딩(Greedy decoding) : 각 단계에 대해 단순히 decoder_output 에서 가장 높은 softmax값을 갖는 단어를 선택하는 방식
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):

        # EncoderRNN의 forward부분 실행
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        #print('outputs : ', encoder_outputs.shape)
        #print('hidden : ', encoder_hidden.shape)
               
        # encoder의 마지막 hidden이 decoder의 처음 hidden
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        #print('decoder_hidden : ', decoder_hidden.shape)
        
        # decoder의 처음입력을 SOS로 초기화
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * CLS_token
        #print('decoder_input : ', decoder_input)
        
        # 디코더가 단어를 덧붙여 나갈 텐서를 초기화
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)

        for _ in range(max_length):
            # LuongAttnDecoderRNN의 forward로 실행
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            #print('decoder_output : ', decoder_output.shape)
            #print('decoder_hidden : ', decoder_hidden.shape)
            
            # 가장 가능성 높은 단어 토큰과 그 softmax 점수를 구합니다
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            #print('decoder_scores : ', decoder_scores)
            #print('decoder_input : ', decoder_input)

            # 토큰, 점수 기록
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)

            # 현재의 토큰을 디코더의 다음 입력으로 준비시킵니다(차원을 증가시켜서)
            decoder_input = torch.unsqueeze(decoder_input, 0)

        return all_tokens, all_scores

In [ ]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    # indexes_batch : 문장을 단어집합에 저장된 수로 바꾼후 마지막에 EOS추가하는 함수
    token_to_encode = transform([sentence])
    indexes_batch = [token_to_encode[0].tolist()]
    #print(indexes_batch)
    #print('input : ', indexes_batch)
 
    # lengths 텐서를 만듭니다
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    #print(lengths)
    
    # 배치의 차원을 뒤집어서 모델이 사용하는 형태로 만듭니다
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    #print('input : ', input_batch)
    
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    
    # searcher를 이용하여 문장을 디코딩합니다
    G_tokens, scores = searcher(input_batch, lengths, max_length)
    #print(G_tokens)
    #print(type(G_tokens))
    #print(scores)

    # 인덱스 -> 단어    
    #print([tokenizer.convert_ids_to_tokens(token) for token in G_tokens])
#     for token in G_tokens:
#         print(tokenizer.convert_ids_to_tokens([token]))
    
    decoded_words = []
    
    for i in G_tokens:
        decoded_words.append(vocab.idx_to_token[i])
        #print(vocab.idx_to_token[i])
        #print(decoded_words)
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # 입력 문장을 받아옵니다
            input_sentence = input('> ')
            # 종료 조건인지 검사합니다
            if input_sentence == 'exit' or input_sentence == '종료': break
            # 문장을 평가합니다
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # 응답 문장을 형식에 맞춰 출력합니다
            output_words[:] = [x for x in output_words if not (x == '[CLS]' or x == '[SEP]' or x == 'PAD' or x == 'SOS' or x == 'EOS')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [ ]:
# Dropout 레이어를 평가 모드로 설정합니다
encoder.eval()
decoder.eval()

# 탐색 모듈을 초기화합니다
searcher = GreedySearchDecoder(encoder, decoder)

# 채팅을 시작합니다 (다음 줄의 주석을 제거하면 시작해볼 수 있습니다)
evaluateInput(encoder, decoder, searcher, voc)